'multi_turn_base': 'MULTI_TURN',

'multi_turn_miss_func': 'MULTI_TURN',

'multi_turn_miss_param': 'MULTI_TURN',

'multi_turn_long_context': 'MULTI_TURN'

In [1]:
data_path='/dfs/data/datasets/bfcl_v3/data/train-00000-of-00001.parquet'

In [2]:
import pandas as pd

In [3]:
df=pd.read_parquet(data_path)

In [6]:
df.to_json("/dfs/data/datasets/bfcl_v3/data.json",orient="records",lines=True)

In [12]:
multi_turn=df[df['multi_turn']==True]

In [28]:
multi_turn.head()

,id,multi_turn,functions,tools,missed_functions,initial_config,involved_classes,turns,language,test_category,subset,ground_truth
3641,multi_turn_base_0,True,"[{""name"": ""authenticate_twitter"", ""description...","[{""type"": ""function"", ""function"": {""name"": ""au...",{},"{""GorillaFileSystem"": {""root"": {""workspace"": {...","[TwitterAPI, GorillaFileSystem]","[[{""role"": ""system"", ""content"": ""\nYou are a h...",Python,multi_turn_base,multi_turn_base,"[[""cd(folder='document')"", ""mkdir(dir_name='te..."
3642,multi_turn_base_1,True,"[{""name"": ""cat"", ""description"": ""This tool bel...","[{""type"": ""function"", ""function"": {""name"": ""ca...",{},"{""GorillaFileSystem"": {""root"": {""alex"": {""type...",[GorillaFileSystem],"[[{""role"": ""system"", ""content"": ""\nYou are a h...",Python,multi_turn_base,multi_turn_base,"[[""ls(a=True)""], [""cd(folder='workspace')"", ""m..."
3643,multi_turn_base_2,True,"[{""name"": ""close_ticket"", ""description"": ""This...","[{""type"": ""function"", ""function"": {""name"": ""cl...",{},"{""GorillaFileSystem"": {""root"": {""simona"": {""ty...","[TicketAPI, GorillaFileSystem]","[[{""role"": ""system"", ""content"": ""\nYou are a h...",Python,multi_turn_base,multi_turn_base,"[[""cd(folder='documents')"", ""touch(file_name='..."
3644,multi_turn_base_3,True,"[{""name"": ""cat"", ""description"": ""This tool bel...","[{""type"": ""function"", ""function"": {""name"": ""ca...",{},"{""GorillaFileSystem"": {""root"": {""alex"": {""type...",[GorillaFileSystem],"[[{""role"": ""system"", ""content"": ""\nYou are a h...",Python,multi_turn_base,multi_turn_base,"[[""find(path='.',name='test')""], [""cd(folder='..."
3645,multi_turn_base_4,True,"[{""name"": ""authenticate_twitter"", ""description...","[{""type"": ""function"", ""function"": {""name"": ""au...",{},"{""GorillaFileSystem"": {""root"": {""tmp"": {""type""...","[TwitterAPI, GorillaFileSystem]","[[{""role"": ""system"", ""content"": ""\nYou are a h...",Python,multi_turn_base,multi_turn_base,"[[""ls(a=True)""], [""sort(file_name='report.txt'..."


In [27]:
multi_turn['turns'].iloc[42]

'[[{"role": "system", "content": "\\nYou are a helpful assistant and an expert in function composition. You can answer general questions using your internal knowledge OR invoke functions when necessary. Follow these strict guidelines:\\n\\n1. FUNCTION CALLS:\\n- ONLY use functions that are EXPLICITLY listed in the function list below\\n- If NO functions are listed (empty function list []), respond ONLY with internal knowledge or \\"I don\'t have access to [Unavailable service] information\\"\\n- If a function is not in the list, respond ONLY with internal knowledge or \\"I don\'t have access to [Unavailable service] information\\"\\n- If ALL required parameters are present AND the query EXACTLY matches a listed function\'s purpose: output ONLY the function call(s)\\n- Use exact format: [func_name1(param1=value1, param2=value2), func_name2(...)]\\nExamples:\\nCORRECT: [get_weather(location=\\"Vancouver\\"), calculate_route(start=\\"Boston\\", end=\\"New York\\")] <- Only if get_weather 

### 查看字符串最长的数据

In [29]:
import json

def estimate_tokens(x):
    try:
        data = json.loads(x) if isinstance(x, str) else x
        if isinstance(data, list) and len(data) > 0 and isinstance(data[0], list):
            conversation = data[0]
        else:
            conversation = data
            
        full_text = ""
        for turn in conversation:
            if turn.get('content'):
                full_text += str(turn.get('content'))
        
        # Qwen 的 token 密度较高，粗略按 1.5 ~ 2 字符 = 1 token 估算 (如果不含大量英文)
        return len(full_text)
    except:
        return 0

# 计算字符数
char_counts = multi_turn['turns'].apply(estimate_tokens)
print(f"字符数最多的一条包含: {char_counts.max()} 字符")

字符数最多的一条包含: 33186 字符


### 确定token最长的数据

In [30]:
from transformers import AutoTokenizer
import json

# 1. 加载 Tokenizer (指向你本地权重的路径，或者 huggingface 上的 id)
# 这里的路径换成你服务器上 Qwen 模型的实际路径
model_path = "/dfs/data/models/Qwen3-0.6B" # 填入你的模型路径
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def calculate_total_tokens(x):
    try:
        # 解析 JSON 字符串
        data = json.loads(x) if isinstance(x, str) else x
        
        # 处理嵌套列表 [[...]] 的情况
        if isinstance(data, list) and len(data) > 0 and isinstance(data[0], list):
            conversation = data[0]
        else:
            conversation = data
            
        # 拼接所有轮次的 content
        # 注意：这只是纯内容的 token，还没算 template 的特殊 token (<|im_start|>等)
        # 如果要非常精确，最好用 tokenizer.apply_chat_template(conversation)
        full_text = ""
        for turn in conversation:
            content = turn.get('content', '')
            # 有些 content 可能是 None
            if content:
                full_text += str(content)
        
        # 计算 token 数量
        return len(tokenizer.encode(full_text))
    except Exception as e:
        return 0

# 应用函数
token_counts = multi_turn['turns'].apply(calculate_total_tokens)

# 打印结果
max_tokens = token_counts.max()
max_idx = token_counts.idxmax()

print(f"🔥 所有数据中，Token 最多的一条包含: {max_tokens} tokens")
print(f"📍 它的索引是: {max_idx}")

ModuleNotFoundError: No module named 'transformers'